In [1]:
import uuid
from googleads import adwords
import sys

In [2]:
from csv import reader
import csv
import pandas as pd
import numpy as np
import copy

In [3]:
AUTH_FILE_PATH = '/home/tim_su/ai_optimizer/opt/ai_optimizer/googleads.yaml'
client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
#client_2 = ad_manager.AdManagerClient.LoadFromStorage('/home/tim_su/ai_optimizer/opt/ai_optimizer/googleads.yaml')

In [4]:
CUSTOMER_SERVICE_ID = '3637290511'
CUSTOMER_SERVICE_ID = '5066360290'
client.SetClientCustomerId(CUSTOMER_SERVICE_ID)

## Get Available Fields of Report Type

In [ ]:
# Display which field is able to query for certain report type
REPORT_TYPE = 'CAMPAIGN_PERFORMANCE_REPORT'

def get_report_fields(client, report_type):
    # Initialize appropriate service.
    report_definition_service = client.GetService(
        'ReportDefinitionService', version='v201809')

    # Get report fields.
    fields = report_definition_service.getReportFields(report_type)

    # Display results.
    print('Report type "%s" contains the following fields:' % report_type)
    for field in fields:
        print(' - %s(%s)' % (field['fieldName'], field['fieldType']))
        if 'enumValues' in field:
            print('  := [%s]' % ', '.join(field['enumValues']))

if __name__ == '__main__':
    # Initialize client object.
    adwords_client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
    get_report_fields(adwords_client, REPORT_TYPE)

## Campaign Insights & AdGroup Insights

In [ ]:
CAMPAIGN_FIELDS = ['ExternalCustomerId','CampaignId','AdvertisingChannelType', 'CampaignStatus',
                   'BiddingStrategyType','Amount','StartDate','EndDate','Cost',
                   'AverageCost','Impressions', 'Clicks','Conversions',
                   'AverageCpc','CostPerConversion', 'Ctr']
class DatePreset:
    today = 'TODAY'
    yesterday = 'YESTERDAY'
    lifetime = 'ALL_TIME'

def get_campaign_insights(client, campaign_id=None, date_preset=None):
    client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
    report_downloader = client.GetReportDownloader(version='v201809')
    if date_preset is None:
        date_preset = 'ALL_TIME'
    else:
        date_preset = date_preset
    # Create report definition.
    report = {
        'reportName': 'CAMPAIGN_PERFORMANCE_REPORT',
#         'dateRangeType': 'CUSTOM_DATE',
        'dateRangeType': date_preset,
        'reportType': 'CAMPAIGN_PERFORMANCE_REPORT',
        'downloadFormat': 'CSV',
        'selector': {
            'fields': CAMPAIGN_FIELDS,
#             'dateRange': {'min': '20190301','max': '20190401'},
            'predicates': [
                {
                    'field': 'CampaignId',
                    'operator': 'EQUALS',
                    'values':[campaign_id]
                }
            ]
        }
    }
    # Print out the report as a string
    csv = report_downloader.DownloadReportAsString(  
        report, skip_report_header=True, skip_column_header=True,   
        skip_report_summary=True, include_zero_impressions=True,client_customer_id=CUSTOMER_SERVICE_ID)
    csv_list = csv.split('\n')[:-1]
    df = pd.DataFrame()
    for lil_csv in csv_list:
        df_temp = pd.DataFrame(
            data=np.array(lil_csv.split(',')).reshape(1,len(CAMPAIGN_FIELDS)),
            columns=CAMPAIGN_FIELDS
        )
        df = df.append(df_temp)
        df['Ctr'] = df.Ctr.str.split('%', expand = True)[0]
    df[df.columns.difference(['CampaignStatus', 'AdvertisingChannelType', 'BiddingStrategyType', 'StartDate', 'EndDate'])] = df[df.columns.difference(
        ['CampaignStatus', 'AdvertisingChannelType', 'BiddingStrategyType', 'StartDate', 'EndDate'])].apply(pd.to_numeric, errors='coerce')
    df[df.columns.difference(['ExternalCustomerId', 'CampaignId', 'AdvertisingChannelType', 'CampaignStatus', 'BiddingStrategyType', 'StartDate', 'EndDate', 'Impressions', 'Clicks', 'Conversions', 'Ctr'])] = df[df.columns.difference(
        ['ExternalCustomerId', 'CampaignId', 'CampaignStatus', 'AdvertisingChannelType', 'BiddingStrategyType', 'StartDate', 'EndDate', 'Impressions', 'Clicks', 'Conversions', 'Ctr'])].div(1000000)
    df[['StartDate','EndDate']] = df[['StartDate','EndDate']].apply( pd.to_datetime, errors='coerce' )
    return df




In [ ]:
if __name__ == '__main__':
    # Initialize client object.
    campaign_id = 1809236789
    client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
    campaign_insights = get_campaign_insights(client, campaign_id=campaign_id)

In [ ]:
campaign_insights

In [ ]:
ADGROUP_FIELDS = ['ExternalCustomerId','CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus',
                  'CpmBid','CpvBid', 'CpcBid', 'TargetCpa', 'BiddingStrategyType','Cost',
                   'AverageCost','Impressions', 'Clicks','Conversions',
                   'AverageCpc','CostPerConversion']
# ADGROUP_FIELDS = ['ExternalCustomerId','CampaignId', 'AdGroupType', 'AdGroupId','ConversionCategoryName']
def get_adgroup_insights(client, campaign_id=None, adgroup_id=None, date_preset=None):
    client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
    report_downloader = client.GetReportDownloader(version='v201809')
    if date_preset is None:
        date_preset = 'ALL_TIME'
    else:
        date_preset = date_preset
    # Create report definition.
    if campaign_id is not None and adgroup_id is None:
        operand = {
                    'field': 'CampaignId',
                    'operator': 'EQUALS',
                    'values':[campaign_id]
        }
    elif adgroup_id is not None:
        operand = {
                    'field': 'AdGroupId',
                    'operator': 'EQUALS',
                    'values':[adgroup_id]
        }
    else:
        print('get_adgroup_insights: Missing arguments campaign_id or adgroup_id.' )
        operand = None
        return
    report = {
        'reportName': 'ADGROUP_PERFORMANCE_REPORT',
#         'dateRangeType': 'CUSTOM_DATE',
        'dateRangeType': date_preset,
        'reportType': 'ADGROUP_PERFORMANCE_REPORT',
        'downloadFormat': 'CSV',
        'selector': {
            'fields': ADGROUP_FIELDS,
#             'dateRange': {'min': '20190301','max': '20190401'},
            'predicates': [
                operand
            ]
        }
    }
    # Print out the report as a string
    csv = report_downloader.DownloadReportAsString(  
        report, skip_report_header=True, skip_column_header=True,   
        skip_report_summary=True, include_zero_impressions=True,client_customer_id=CUSTOMER_SERVICE_ID)
    csv_list = csv.split('\n')[:-1]
    df = pd.DataFrame()
    for lil_csv in csv_list:
        df_temp = pd.DataFrame(
            data=np.array(lil_csv.split(',')).reshape(1,len(ADGROUP_FIELDS)),
            columns=ADGROUP_FIELDS
        )
        df = df.append(df_temp)
    df[df.columns.difference(['AdGroupType', 'AdGroupStatus', 'BiddingStrategyType'])] = df[df.columns.difference(
        ['AdGroupType', 'AdGroupStatus', 'BiddingStrategyType'])].apply(pd.to_numeric, errors='coerce')
    df[df.columns.difference( ['ExternalCustomerId', 'CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'BiddingStrategyType', 'Conversions'] )] = df[df.columns.difference(
        ['ExternalCustomerId', 'CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'BiddingStrategyType', 'Conversions'])].div(1000000)

    return df.reset_index(drop=True)

if __name__ == '__main__':
    # Initialize client object.
    client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
#     adgroup_insights = get_adgroup_insights(client)
    adgroup_insights = get_adgroup_insights(client, campaign_id=campaign_id)

## Retrieve Age Range Performance Report

In [ ]:
AGE_RANGE_FIELDS = ['ExternalCustomerId','CampaignId', 'AdGroupId','Criteria', 'Id', 'AdGroupStatus',
                  'CpmBid', 'CpcBid', 'BiddingStrategyType','Cost',
                   'AverageCost','Impressions', 'Clicks','Conversions',
                   'AverageCpc','CostPerConversion']
# AGE_RANGE_FIELDS = ['ExternalCustomerId','CampaignId', 'AdGroupType', 'AdGroupId','ConversionCategoryName']
def get_age_range_insights(client, campaign_id=None, adgroup_id=None, date_preset=None):
    client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
    report_downloader = client.GetReportDownloader(version='v201809')
    if date_preset is None:
        date_preset = 'ALL_TIME'
    else:
        date_preset = date_preset
    # Create report definition.
    if campaign_id is not None and adgroup_id is None:
        operand = {
                    'field': 'CampaignId',
                    'operator': 'EQUALS',
                    'values':[campaign_id]
        }
    elif adgroup_id is not None:
        operand = {
                    'field': 'AdGroupId',
                    'operator': 'EQUALS',
                    'values':[adgroup_id]
        }
    else:
        print('get_age_range_insights: Missing arguments campaign_id or adgroup_id.' )
        operand = None
        return
    report = {
        'reportName': 'AGE_RANGE_PERFORMANCE_REPORT',
        'dateRangeType': date_preset,
        'reportType': 'AGE_RANGE_PERFORMANCE_REPORT',
        'downloadFormat': 'CSV',
        'selector': {
            'fields': AGE_RANGE_FIELDS,
            'predicates': [
                operand
            ]
        }
    }
    # Print out the report as a string
    csv = report_downloader.DownloadReportAsString(  
        report, skip_report_header=True, skip_column_header=True,   
        skip_report_summary=True, include_zero_impressions=True,client_customer_id=CUSTOMER_SERVICE_ID)
    csv_list = csv.split('\n')[:-1]
    df = pd.DataFrame()
    for lil_csv in csv_list:
        df_temp = pd.DataFrame(
            data=np.array(lil_csv.split(',')).reshape(1,len(AGE_RANGE_FIELDS)),
            columns=AGE_RANGE_FIELDS
        )
        df = df.append(df_temp)
    df[df.columns.difference(['AdGroupType', 'AdGroupStatus', 'BiddingStrategyType', 'Criteria'])] = df[df.columns.difference(
        ['AdGroupType', 'AdGroupStatus', 'BiddingStrategyType', 'Criteria'])].apply(pd.to_numeric, errors='coerce')
    df[df.columns.difference( ['ExternalCustomerId', 'CampaignId','Criteria', 'Id', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'BiddingStrategyType','Impressions', 'Clicks', 'Conversions'] )] = df[df.columns.difference(
        ['ExternalCustomerId', 'CampaignId', 'Criteria', 'Id', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'BiddingStrategyType','Impressions', 'Clicks', 'Conversions'])].div(1000000)

    return df.reset_index(drop=True)

# if __name__ == '__main__':
# Initialize client object.
client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
#     adgroup_insights = get_age_range_insights(client)
adgroup_insights = get_age_range_insights(client, campaign_id=1747836670, adgroup_id=68524904775)

## Retrieve Performance Report from Different Criterions

In [ ]:
# URL
URL_FIELDS_LIST = [
    'ExternalCustomerId', 'CampaignId', 'AdGroupId', 'AdGroupStatus', 'Cost', 'AverageCost',
    'Impressions', 'Clicks', 'Conversions', 'AverageCpc', 'CostPerConversion', 'DisplayName', ]

# CRITERIA,
CRITERIA_FIELDS_LIST = [
    'ExternalCustomerId', 'CampaignId', 'AdGroupId', 'Criteria', 'Id', 'AdGroupStatus', 'CpmBid', 'CpcBid',
    'Cost', 'AverageCost', 'Impressions', 'Clicks', 'Conversions', 'AverageCpc', 'CostPerConversion']

# AUDIENCE, AGE_RANGE, DISPLAY_KEYWORD
BIDDABLE_FIELDS_LIST = [
    'ExternalCustomerId', 'CampaignId', 'AdGroupId', 'Criteria', 'Id', 'AdGroupStatus', 'CpmBid', 'CpcBid',
    'BiddingStrategyType', 'Cost', 'AverageCost', 'Impressions', 'Clicks', 'Conversions', 'AverageCpc', 'CostPerConversion']

# ADGROUP
ADGROUP_FIELDS_LIST = [
    'ExternalCustomerId', 'CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'CpmBid', 'CpvBid', 'CpcBid', 'TargetCpa',
    'BiddingStrategyType', 'Cost', 'AverageCost', 'Impressions', 'Clicks', 'Conversions', 'AverageCpc', 'CostPerConversion']

NON_NUMERIC_FIELDS_LIST = [
    'Ad group type', 'Ad group state', 'Bid Strategy Type', 'Keyword', 'Age Range', 'Audience', 'Keyword / Placement', 'Criteria Display Name']

NUMERIC_FIELDS_LIST = [
    'Max. CPM', 'Max. CPV', 'Max. CPC', 'Default max. CPC', 'Target CPA', 'Cost', 'Avg. Cost', 'Avg. CPC', 'Cost / conv.']

FIELD_INDEX = {
    'ADGROUP': ADGROUP_FIELDS_LIST,
    'URL': URL_FIELDS_LIST,
    'CRITERIA': CRITERIA_FIELDS_LIST,
    'AUDIENCE': BIDDABLE_FIELDS_LIST,
    'AGE_RANGE': BIDDABLE_FIELDS_LIST,
    'DISPLAY_KEYWORD': BIDDABLE_FIELDS_LIST,
}

ADGROUP_FIELDS_LIST  = [
    'customer_id', 'campaign_id', 'channel_type', 'adgroup_id', 'status', 'cpm_bid', 'cpv_bid', 'cpc_bid', 'cpa_bid', 'bidding_type', 'spend', 'cost_per_target', 'impressions', 'clicks', 'conversions', 'cost_per_click', 'cost_per_conversion']
URL_FIELDS_LIST      = [
    'customer_id', 'campaign_id', 'adgroup_id', 'status', 'spend', 'cost_per_target', 'impressions', 'clicks', 'conversions', 'cost_per_click', 'cost_per_conversion', 'url_display_name']
CRITERIA_FIELDS_LIST = [
    'customer_id', 'campaign_id', 'adgroup_id', 'keyword_placement', 'keyword_id', 'status', 'cpm_bid', 'cpc_bid', 'spend', 'cost_per_target', 'impressions', 'clicks', 'conversions', 'cost_per_click', 'cost_per_conversion']
AUDIENCE_FIELDS_LIST = [
    'customer_id', 'campaign_id', 'adgroup_id', 'audience', 'criterion_id', 'status', 'cpm_bid', 'cpc_bid', 'bidding_type', 'spend', 'cost_per_target', 'impressions', 'clicks', 'conversions', 'cost_per_click', 'cost_per_conversion']
AGE_RANGE_FIELDS_LIST = [
    'customer_id', 'campaign_id', 'adgroup_id', 'age_range', 'criterion_id', 'status', 'cpm_bid', 'cpc_bid', 'bidding_type', 'spend', 'cost_per_target', 'impressions', 'clicks', 'conversions', 'cost_per_click', 'cost_per_conversion']
BIDDABLE_FIELDS_LIST = [
    'customer_id', 'campaign_id', 'adgroup_id', 'keyword', 'keyword_id', 'status', 'cpm_bid', 'cpc_bid', 'bidding_type', 'spend', 'cost_per_target', 'impressions', 'clicks', 'conversions', 'cost_per_click', 'cost_per_conversion']

COLUMN_INDEX = {
    'ADGROUP': ADGROUP_FIELDS_LIST,
    'URL': URL_FIELDS_LIST,
    'CRITERIA': CRITERIA_FIELDS_LIST,
    'AUDIENCE': AUDIENCE_FIELDS_LIST,
    'AGE_RANGE': AGE_RANGE_FIELDS_LIST,
    'DISPLAY_KEYWORD': BIDDABLE_FIELDS_LIST,
}
DATABASE_INDEX = {
    'ADGROUP': 'adgroup_insights',
    'URL': 'url_insights',
    'CRITERIA': 'criteria_insights',
    'AUDIENCE': 'audience_insights',
    'AGE_RANGE': 'age_range_insights',
    'DISPLAY_KEYWORD': 'display_keyword_insights',
}

In [ ]:

def get_performance_insights(client, campaign_id=None, adgroup_id=None, date_preset=None, performance_type='ADGROUP'):
    client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
    report_downloader = client.GetReportDownloader(version='v201809')
    if date_preset is None:
        date_preset = 'ALL_TIME'
    # Create report definition.
    if campaign_id is not None and adgroup_id is None:
        operand = {
            'field': 'CampaignId',
            'operator': 'EQUALS',
            'values': [campaign_id]
        }
    elif adgroup_id is not None:
        operand = {
            'field': 'AdGroupId',
            'operator': 'EQUALS',
            'values': [adgroup_id]
        }
    else:
        print('get_audience_insights: Missing arguments campaign_id or adgroup_id.')
        operand = None
        return
    report = {
        'downloadFormat': 'CSV',
        'reportName': performance_type+'_PERFORMANCE_REPORT',
        'reportType': performance_type+'_PERFORMANCE_REPORT',
        'dateRangeType': date_preset,
        'selector': {
            'fields': FIELD_INDEX[performance_type],
            'predicates': [operand]
        }
    }
    # Print out the report as a string
    with open(performance_type+'.csv', 'wb') as output_file:
        report_downloader.DownloadReport(
            report, output=output_file, skip_report_header=True, skip_column_header=False,
            skip_report_summary=True, include_zero_impressions=False, client_customer_id=CUSTOMER_SERVICE_ID)
    with open(performance_type+'.csv')as csv_file:
        df = pd.read_csv(csv_file, sep=",", quotechar='"')
        
        df[df.columns.difference( NON_NUMERIC_FIELDS_LIST )] = df[df.columns.difference( NON_NUMERIC_FIELDS_LIST )].apply(pd.to_numeric, errors='coerce')
        
        df[df.columns.intersection( NUMERIC_FIELDS_LIST )] = df[df.columns.intersection( NUMERIC_FIELDS_LIST )].div(1000000)
        
        df.columns = COLUMN_INDEX[performance_type]
        gdn_db.into_table( df, DATABASE_INDEX[performance_type] )
        return df.sort_values(by=['impressions'], ascending=False).reset_index(drop=True)


if __name__ == '__main__':
    # Initialize client object.
    # URL
    # CRITERIA
    # AUDIENCE, AGE_RANGE, DISPLAY_KEYWORD
    from ai_optimizer.codes import gdn_db
    client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
    display_keyword_insights = get_performance_insights(
        client, campaign_id=1747836664, adgroup_id=68524904535, performance_type='DISPLAY_KEYWORD')

## Make AdGroup

In [ ]:
from googleads import adwords
CUSTOMER_SERVICE_ID = '3637290511'
client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
AD_GROUP_ID = 68524904055
CPC_BID_MICRO_AMOUNT = '4000000'


def update_adgroup_bid(client, ad_group_id, bid_micro_amount=None):
    # Initialize appropriate service.
    ad_group_service = client.GetService('AdGroupService', version='v201809')
    
    # Construct operations and update an ad group.
    operations = [{
        'operator': 'SET',
        'operand': {
            'id': ad_group_id,
            'status': 'ENABLED'
        }
    }]
  
    if bid_micro_amount:
        operations[0]['operand']['biddingStrategyConfiguration'] = {
            'bids': [{
                'xsi_type': 'CpcBid',
                'bid': {
                    'microAmount': bid_micro_amount,
                }
            }]
        }
  
    ad_groups = ad_group_service.mutate(operations)
    return ad_groups

if __name__ == '__main__':
    # Initialize client object.
    adwords_client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
    adwords_client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
    update_adgroup_bid(adwords_client, AD_GROUP_ID, CPC_BID_MICRO_AMOUNT)

In [ ]:

adwords_client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
adwords_client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
update_adgroup_bid(adwords_client, AD_GROUP_ID, CPC_BID_MICRO_AMOUNT)

In [ ]:
def retrieve_campaign_params(client, campaign_id):
    _FIELDS = ['CampaignId', 'Name']
    selector= [{
        'fields': _FIELDS,
    #             'dateRange': {'min': '20190301','max': '20190401'},
        'predicates': [
            {
                'field': 'CampaignId',
                'operator': 'EQUALS',
                'values':[campaign_id]
            }
        ]
    }]
    campaign_service = client.GetService('CampaignService', version='v201809')
    campaign_params = campaign_service.get(selector)
    return campaign_params

## Retrive AdGroup Params

### Params

#### AdGroup

In [ ]:
def retrieve_ad_group_params(client, ad_group_id):
    _FIELDS = ['CampaignId', 'AdGroupId', 'Name', 'CpcBid', 'BiddingStrategyId','BiddingStrategyName','BiddingStrategySource', 'BiddingStrategyType']
    selector= [{
        'fields': _FIELDS,
    #             'dateRange': {'min': '20190301','max': '20190401'},
        'predicates': [
            {
                'field': 'AdGroupId',
                'operator': 'EQUALS',
                'values':[ad_group_id]
            }
        ]
    }]
    ad_group_service = client.GetService('AdGroupService', version='v201809')
    ad_group_params = ad_group_service.get(selector)
    return ad_group_params

def make_ad_group(ad_group_id):
    ad_group_params = retrieve_ad_group_params(client, ad_group_id)
    campaign_id = ad_group_params['entries'][0]['campaignId']
    name = ad_group_params['entries'][0]['name']
    xsi_type = ad_group_params['entries'][0]['biddingStrategyConfiguration']['bids'][0]['Bids.Type']
    micro_amount = ad_group_params['entries'][0]['biddingStrategyConfiguration']['bids'][0]['bid']['microAmount']
    operations = [{
        'operator': 'ADD',
        'operand': {
            'campaignId': campaign_id,
            'name':  'Mutant {}'.format(name),
            'status': 'ENABLED',
            'biddingStrategyConfiguration': {
                'bids': [
                    {
                        'xsi_type': xsi_type,
                        'bid': {
                            'microAmount': micro_amount
                        }
                    }
                ]
            }
        }
    }]
    ad_group_service = client.GetService('AdGroupService', version='v201809')
    ad_groups = ad_group_service.mutate(operations)
    return ad_groups

#### Ad

In [ ]:
def retrive_ad_params(ad_group_id):
    ad_params_list = []
    ad_criteria = dict()
    _FIELDS = ['AdGroupId', 'HeadlinePart1', 'HeadlinePart2', 'DisplayUrl', 'CreativeFinalUrls', 'Description', 'Url']
    selector= [{
        'fields': _FIELDS,
        'predicates': [
            {
                'field': 'AdGroupId',
                'operator': 'EQUALS',
                'values':[ad_group_id]
            }
        ]
    }]
    ad_service = client.GetService('AdGroupAdService', version='v201809')
    ad_params = ad_service.get(selector)
    for entry in ad_params['entries']:
        ad_criteria['id'] = entry['ad']['id']
        ad_params_list.append(copy.deepcopy(ad_criteria))
    return ad_params_list

def assign_ad_to_ad_group(ad_group_id, ad_params_list):
    operations = []
    for ad_params in ad_params_list:
        operations.append({
            'operator': 'ADD',
            'operand': {
                'adGroupId': ad_group_id,
                'status': 'ENABLED',
                'ad': ad_params
            }
        })
    ad_group_service = client.GetService('AdGroupAdService', version='v201809')
    ad_groups = ad_group_service.mutate(operations)
    return ad_groups

### Criterion

In [ ]:
def retrieve_ad_group_criterion(client, ad_group_id):
    _FIELDS = ['AdGroupId', 'CriteriaType', 'UserInterestId', 'UserInterestName', 'UserListId', 'LabelIds']
    selector= [{
        'fields': _FIELDS,
    #             'dateRange': {'min': '20190301','max': '20190401'},
        'predicates': [
            {
                'field': 'AdGroupId',
                'operator': 'EQUALS',
                'values':[ad_group_id]
            }
        ]
    }]
    ad_group_service = client.GetService('AdGroupCriterionService', version='v201809')
    ad_group_criterion = ad_group_service.get(selector)
    return ad_group_criterion

def make_adgroup_criterion(new_adgroup_id = None, old_ad_group_criterion=None):
    if new_adgroup_id is None and old_ad_group_criterion is None:
        raise ValueError('new_adgroup_id and old_ad_group_criterion is Required.')
    biddable_criterion = dict()
    negative_criterion = dict()
    biddable_criterion['adGroupId'] = new_adgroup_id
    negative_criterion['adGroupId'] = new_adgroup_id

    biddable_ad_group_criteria_list = list()
    negative_ad_group_criteria_list = list()
    for entry in old_ad_group_criterion['entries']:
        criterion = dict()
        
        if 'AgeRange' in entry['criterion']['Criterion.Type']:
            criterion['id'] = entry['criterion']['id']
            criterion['xsi_type'] = entry['criterion']['Criterion.Type']
            
        elif 'Placement'  in entry['criterion']['Criterion.Type']:
            criterion['id'] = entry['criterion']['id']
            criterion['url'] = entry['criterion']['url']
            criterion['xsi_type'] = entry['criterion']['Criterion.Type']

        elif 'Keyword'  in entry['criterion']['Criterion.Type']:
            criterion['id'] = entry['criterion']['id']
            criterion['matchType'] = entry['criterion']['matchType']
            criterion['text'] = entry['criterion']['text']
            criterion['xsi_type'] = entry['criterion']['Criterion.Type']

        elif 'CriterionUserInterest'  in entry['criterion']['Criterion.Type']:
            criterion['id'] = entry['criterion']['id']
            criterion['userInterestId'] = entry['criterion']['userInterestId']
            criterion['xsi_type'] = entry['criterion']['Criterion.Type']
        else:
            pass
        if criterion:
            if 'BiddableAdGroupCriterion' in entry['AdGroupCriterion.Type']:
                biddable_criterion['xsi_type'] = entry['AdGroupCriterion.Type']
                biddable_criterion['criterion'] = criterion
                biddable_ad_group_criteria_list.append(copy.deepcopy(biddable_criterion))

            elif 'NegativeAdGroupCriterion' in entry['AdGroupCriterion.Type']:
                negative_criterion['xsi_type'] = entry['AdGroupCriterion.Type']
                negative_criterion['criterion'] = criterion
                negative_ad_group_criteria_list.append(copy.deepcopy(negative_criterion))
    return biddable_ad_group_criteria_list, negative_ad_group_criteria_list
 

def assign_criterion_to_ad_group(client, ad_group_id, ad_group_criterion):
    '''
    an example of addong 4 type of criterion to new adgroup,
    if trying to exclude certain criterions, 
    change 'xsi_type' from 'BiddableAdGroupCriterion' to 'NegativeAdGroupCriterion' in ad_group_criteria object
    '''
    # Initialize appropriate service.
    ad_group_criterion_service = client.GetService(
        'AdGroupCriterionService', version='v201809')
    # Create the ad group criteria.  
    # Create operations.
    operations = []
    for criterion in ad_group_criterion:
        operations.append({
            'operator': 'ADD',
            'operand': criterion
        })
    response = ad_group_criterion_service.mutate(operations)
    return response

def make_adgroup_criterion_by_score(campaign_id, new_ad_group_id):
    criterions=[]
    for criteria in SCORE_INDEX.keys():
        df = gdn_db.get_table(campaign_id=campaign_id, table=SCORE_INDEX[criteria]+"_score").head()
        if not df.empty:
#             print(criteria)
            sub_criterions = []
            df = df.sort_values(by=['score'], ascending=False).reset_index(drop=True).drop_duplicates([CRITERIA_COLUMN_INDEX[criteria]])
            criterion = {
                'adGroupId': new_ad_group_id,
                'xsi_type': 'BiddableAdGroupCriterion',
                'criterion': {
                    'xsi_type': None,
                    'id': None
                }
            }
            if criteria == 'AGE_RANGE':
                criterion['criterion']['xsi_type'] = CRITERIA_TYPE_INDEX[criteria]
                for i, criterion_id in enumerate( df[CRITERIA_COLUMN_INDEX[criteria]] ):
                    if CRITERIA_TYPE_INDEX[criteria] == 'AgeRange':
                        criterion['criterion']['id'] = criterion_id
                        sub_criterions.append(copy.deepcopy(criterion))
            else:
                criterion['criterion']['xsi_type'] = CRITERIA_TYPE_INDEX[criteria]
                for i, criterion_id in enumerate( df[CRITERIA_COLUMN_INDEX[criteria]] ):
                    if CRITERIA_TYPE_INDEX[criteria] == 'Placement':
                        criterion['criterion']['id'] = criterion_id
                        criterion['criterion']['url'] = df['url_display_name'].iloc[0]

                    elif CRITERIA_TYPE_INDEX[criteria] == 'Keyword':
#                         criterion['criterion']['id'] = criterion_id
                        criterion['criterion']['matchType'] = 'BROAD'
                        criterion['criterion']['text'] = df['keyword'].iloc[i]

                    elif CRITERIA_TYPE_INDEX[criteria] == 'CriterionUserInterest':
                        criterion['criterion']['id'] = criterion_id
                        criterion['criterion']['userInterestId'] = criterion_id
                    sub_criterions.append(copy.deepcopy(criterion))
            criterions = criterions+sub_criterions
    return criterions

## Make New AdGroup With Criterion From Old AdGroup

In [ ]:
def make_adgroup_with_criterion(old_ad_group_id):
    # Params
    ad_group_params = make_ad_group(old_ad_group_id)
    new_ad_group_id = ad_group_params['value'][0]['id']
    # Criterion
#     old_ad_group_criterion = retrieve_ad_group_criterion(client, old_ad_group_id)
#     biddable_criterion, negative_criterion = make_adgroup_criterion(new_ad_group_id, old_ad_group_criterion)
#     print(biddable_criterion)
#     new_ad_group_criterion = biddable_criterion + negative_criterion
    new_ad_group_criterion = make_adgroup_criterion_by_score(campaign_id, new_ad_group_id)
    result = assign_criterion_to_ad_group(client, new_ad_group_id, new_ad_group_criterion)
    # Assign Ad Params
    ad_params_list = retrive_ad_params(old_ad_group_id)
    assign_ad_to_ad_group(new_ad_group_id, ad_params_list)
    return result

## Test Custom Affinity

In [6]:
custom_affinity_ad_group_id = 68524903895
similar_audience_ad_group_id = 68524904615
ad_group_id = 70749935918

### Criterion

In [ ]:
_FIELDS = ['AdGroupId', 'CriteriaType', 'UserInterestId', 'UserInterestName', 'UserListId', 'LabelIds','CustomAffinityId',
           'FinalUrls', 'UrlCustomParameters', 'CriteriaSamples','UserListMembershipStatus']
selector= [{
    'fields': _FIELDS,
#             'dateRange': {'min': '20190301','max': '20190401'},
    'predicates': [
        {
            'field': 'AdGroupId',
            'operator': 'EQUALS',
            'values':[ad_group_id]
        }
    ]
}]
ad_group_service = client.GetService('AdGroupCriterionService', version='v201809')
ad_group_criterion = ad_group_service.get(selector)


In [ ]:
ad_group_criterion

In [ ]:
for criterion in ad_group_criterion['entries']:
    if criterion['criterion']['type']=='CUSTOM_AFFINITY':
        print(criterion)

In [ ]:
custom_affinity_id = 2511435
id = 456069648865

### Params

In [ ]:
_FIELDS = ['CampaignId', 'AdGroupId', 'Name', 'CpcBid', 'BiddingStrategyId','BiddingStrategyName','BiddingStrategySource',
           'BiddingStrategyType', 'FinalUrlSuffix', 'Settings', 'UrlCustomParameters', 'TrackingUrlTemplate']
selector= [{
    'fields': _FIELDS,
#             'dateRange': {'min': '20190301','max': '20190401'},
    'predicates': [
        {
            'field': 'AdGroupId',
            'operator': 'EQUALS',
            'values':[ad_group_id]
        }
    ]
}]
ad_group_service = client.GetService('AdGroupService', version='v201809')
ad_group_params = ad_group_service.get(selector)

In [ ]:
ad_group_params

### Custom Affinity Service(get url by CustomAffinityId)

In [ ]:
_FIELDS = ['CustomAffinityId', 'Name', 'Status', 'Type']
selector= [{
    'fields': _FIELDS,
#             'dateRange': {'min': '20190301','max': '20190401'},
    'predicates': [
        {
            'field': 'CustomAffinityId',
            'operator': 'EQUALS',
            'values':[custom_affinity_id]
        }
    ]
}]
ad_group_service = client.GetService('CustomAffinityService', version='v201809')
ad_group_params = ad_group_service.get(selector)

In [ ]:
ad_group_params

In [7]:
URL_FIELDS_LIST = [
#     'Criteria','CriteriaDestinationUrl','CriterionAttachmentLevel', 'CustomerDescriptiveName', 'ExternalCustomerId',
#     'FinalAppUrls','FinalMobileUrls', 'FinalUrls','Id','IsRestrict','Status','TrackingUrlTemplate','UrlCustomParameters',
#     'UserListName'
    'ExternalCustomerId', 'CampaignId', 'AdGroupId', 'AdGroupStatus', 'Cost', 'AverageCost',
    'Impressions', 'Clicks', 'Conversions', 'AverageCpc', 'CostPerConversion',
    'VerticalId', 'CustomerDescriptiveName', 'Criteria'
    
]
# client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
report_downloader = client.GetReportDownloader(version='v201809')

operand = {
    'field': 'AdGroupId',
    'operator': 'EQUALS',
    'values': [ad_group_id]
}

report = {
    'downloadFormat': 'CSV',
    'reportName': 'DISPLAY_TOPICS_PERFORMANCE_REPORT',
    'reportType': 'DISPLAY_TOPICS_PERFORMANCE_REPORT',
    'dateRangeType': 'ALL_TIME',
    'selector': {
        'fields': URL_FIELDS_LIST,
        'predicates': [operand]
    }
}
# Print out the report as a string
with open('url_test.csv', 'wb') as output_file:
    report_downloader.DownloadReport(
        report, output=output_file, skip_report_header=True, skip_column_header=False,
        skip_report_summary=True, include_zero_impressions=False, client_customer_id=CUSTOMER_SERVICE_ID)
with open('url_test.csv')as csv_file:
    df = pd.read_csv(csv_file, sep=",", quotechar='"')

In [8]:
df.head()

,Customer ID,Campaign ID,Ad group ID,Ad group state,Cost,Avg. Cost,Impressions,Clicks,Conversions,Avg. CPC,Cost / conv.,Vertical ID,Client name,Topic
0,5066360290,1837064053,70749935918,enabled,378621243,3129101,48595,121,0.0,3129101,0,78,三菱電機,Consumer Electronics
